# Erisoglu 2011 "New" Algorithm

Brief description of task

## Initial setup and configuration

In [1]:
import imports
import sklearn.datasets as skdatasets
#import sklearn.cluster as skcluster
import sklearn.metrics as skmetrics    
import kmeans
import utils
from initialisations import erisoglu

In [16]:
eris = erisoglu.Erisoglu()

dataset = skdatasets.load_iris()
data = dataset.data
target = dataset.target

K = 3

## A few inpections

In [17]:
main = eris._find_main_axis(data.T)
secondary = eris._find_secondary_axis(data.T, main)

print("Axes are: [%i, %i]" % (main, secondary))

Axes are: [3, 1]


In [21]:
initial, axes = eris._initialise(dataset.data)
center = eris._find_center(data, axes)

print("Center was:", center)
print("Initial seed is:", initial)

Center was: [2.3499999999999996, 2.375]
Initial seed is: 15


## Main execution

### Find seeds

In [15]:
seeds = eris.generate(dataset.data, K)

print(seeds)

[[5.7 4.4 1.5 0.4]
 [7.7 2.6 6.9 2.3]
 [4.5 2.3 1.3 0.3]]


### Run k-means

In [20]:
Z, U, clusters, iterations = kmeans.cluster(data, K, seeds.copy())

print(U)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 1 1 1 2 1 1 1 1
 1 1 2 2 1 1 1 1 2 1 2 1 2 1 1 2 2 1 1 1 1 1 2 1 1 1 1 2 1 1 1 2 1 1 1 2 1
 1 2]
